# ProRail Storing Analyse & Voorspellingen
Voor het project Data-driven Business hebben wij de opdracht gekregen om ProRail te helpen bij het voorspellen van de hersteltijd van een storing. We hebben van ProRail een dataset gekregen waar alles in staat over storingen in het verleden. Nu is het aan ons om de data op de juiste manier te analyseren en er voorspellende modellen bij te maken.

## Analyse
Het is erg belangrijk om de data grondig te analysereren. We hebben op dit moment nog geen idee waar we mee werken, dus gaan we dat uitzoeken.

### Inladen & Configureren
Voor dat we aan de analyse gaan beginnen moeten de benodigdheden worden geïmporteerd en geconfigureerd worden. Pandas en Numpy zijn ervoor om de data te analyseren. SciKit-Learn is er voor om de modellen te maken, en Joblib gebruiken we aan het einde om de modellen op te slaan. Daarnaast zijn er nog een aantal configuraties die gedaan moeten worden om het proces soepeler te laten verlopen.

#### Imports

In [39]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import joblib

#### Configuratie

In [40]:
pd.set_option('display.max_columns', None)

in de data zijn values in een column met verschillende Type, met low_memory=False kunt pandas bij inlezen een faste Type voor een column vast leggen.

In [41]:
df = pd.read_csv('Data/sap_storing_data_hu_project.csv', index_col=0, low_memory=False)
df.head()

,#stm_sap_meldnr,stm_mon_nr,stm_vl_post,stm_sap_meld_ddt,stm_sap_mon_meld_ddt,stm_sap_meldtekst_lang,stm_mon_begin_ddt,stm_sap_meldtekst,stm_mon_toelichting_trdl,stm_oh_pg_mld,stm_geo_mld,stm_geo_mld_uit_functiepl,stm_equipm_nr_mld,stm_equipm_soort_mld,stm_equipm_omschr_mld,stm_km_van_mld,stm_km_tot_mld,stm_prioriteit,stm_scenario_mon,stm_status_melding_sap,stm_mon_nr_status_omschr,stm_mon_nr__statuscode,stm_mon_nr_status_wijzdd,stm_aanngeb_ddt,stm_aanntpl_ddt,stm_oh_pg_gst,stm_geo_gst,stm_geo_gst_uit_functiepl,stm_equipm_nr_gst,stm_equipm_soort_gst,stm_equipm_omschr_gst,stm_objectdl_code_gst,stm_objectdl_groep_gst,stm_km_van_gst,stm_km_tot_gst,stm_progfh_in_ddt,stm_progfh_in_invoer_ddt,stm_progfh_gw_ddt,stm_progfh_gw_lwd_ddt,stm_progfh_hz,stm_oorz_groep,stm_oorz_code,stm_oorz_tkst,stm_veroorz_groep,stm_veroorz_code,stm_veroorz_tekst_kort,stm_effect,stm_afspr_aanvangddt,stm_fh_ddt,stm_fh_status,stm_sap_storeind_ddt,stm_mon_eind_ddt,stm_mon_vhdsincident,stm_tao_indicator,stm_tao_indicator_vorige,stm_tao_soort_mutatie,stm_tao_telling_mutatie,stm_tao_beinvloedbaar_indicator,stm_evb,stm_dir_betrok_tr,stm_aangelegd_dd,stm_aangelegd_tijd,stm_sap_melddatum,stm_sap_meldtijd,stm_mon_begindatum,stm_mon_begintijd,stm_contractgeb_mld,stm_functiepl_mld,stm_techn_mld,stm_contractgeb_gst,stm_functiepl_gst,stm_techn_gst,stm_aanngeb_dd,stm_aanngeb_tijd,stm_aanntpl_dd,stm_aanntpl_tijd,stm_arbeid,stm_progfh_in_datum,stm_progfh_in_tijd,stm_progfh_in_invoer_dat,stm_progfh_in_invoer_tijd,stm_progfh_in_duur,stm_progfh_gw_datum,stm_progfh_gw_tijd,stm_progfh_gw_lwd_datum,stm_progfh_gw_lwd_tijd,stm_progfh_gw_duur,stm_progfh_gw_teller,stm_afspr_aanvangdd,stm_afspr_aanvangtijd,stm_fh_dd,stm_fh_tijd,stm_fh_duur,stm_reactie_duur,stm_sap_storeinddatum,stm_sap_storeindtijd,stm_mon_eind_datum,stm_mon_eind_tijd,stm_controle_dd,stm_akkoord_mon_toewijz,stm_status_sapnaarmon,stm_fact_jn,stm_akkoord_melding_jn,stm_afsluit_ddt,stm_afsluit_dd,stm_afsluit_tijd,stm_rec_toegev_ddt,stm_hinderwaarde,stm_actie,stm_standplaats,stm_status_gebr,stm_wbi_nummer,stm_projnr,stm_oorz_tekst_kort,stm_historie_toelichting,stm_schade_verhaalb_jn,stm_schadenr,stm_schade_status_ga,stm_schade_statusdatum,stm_relatiervo_vorig,stm_relatiervo_volgend,stm_relatiervo,stm_pplg_van,stm_pplg_naar,stm_dstrglp_van,stm_dstrglp_naar,stm_afspr_func_hersteldd,stm_afspr_func_hersteltijd,stm_sorteerveld,stm_rapportage_maand,stm_rapportage_jaar,stm_x_bron_publ_dt,stm_x_bron_bestandsnaam,stm_x_bron_arch_dt,stm_x_actueel_ind,stm_x_run_id,stm_x_bk,stm_x_start_sessie_dt,stm_x_vervallen_ind
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,0,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2014,07/05/2014 05:30:12,\\PUHAPS0149\Informatica_Prod\Inbox\Informatic...,31/12/9999 00:00:00,1,34415,3617016,07/05/2014 05:44:24,0
1,50053211,0.0,NaN,02/01/2006 09:00:00,02/01/2006 09:00:00,Logboeknr Geeltje : 49 Tijd: 0900 VL-Po...,NaN,Logboeknr Geeltje : 49 Tijd: 0900,NaN,NaN,624.0,624.0,NaN,NaN,NaN,0.0,0.0,9.0,NaN,MAFS MAFD,NaN,NaN,00/00/0000,02/01/2006 09:00:00,NaN,81S,624.0,624.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,Z,ONR-RIB,133.0,Papieren ontbreken,ONREGRIB,B,Volker,NaN,NaN,02/01/2006 09:00:00,4.0,02/01/2006 09:00:00,NaN,NaN,N,NaN,NaN,0,B,"0,00",NaN,03/01/2006,04:48:18,02/01/2006,09:00:00,NaN,NaN,NaN,624,NaN,81.0,624,S,02/01/2006,09:00:00,NaN,00:00:00,99999999.0,NaN,00:00:00,NaN,00:00:00,99999999.0,NaN,00:00:00,NaN,NaN,99999999.0,0.0,00/00/0000,00:00:00,02/01/2006,09:00:00,0.0,99999999.0,02/01/2006,09:00:00,NaN,NaN,05/01/2006,J,NaN,N,J,05/01/2009 15:10:09,05/01/2009,15:10:09,02/01/2006 04:48:18,0.0,NaN,N

Er zijn een aantal kolommen waar helemaal geen waardes in staan. Die gaan we er dus uit filteren, aangeien we er niks aan hebben.

In [42]:
df = df[[
    "stm_mon_nr", "stm_vl_post", "stm_sap_meld_ddt", "stm_sap_meldtekst_lang",
    "stm_sap_meldtekst", "stm_geo_mld", "stm_equipm_nr_mld", "stm_equipm_soort_mld",
    "stm_equipm_omschr_mld", "stm_km_van_mld", "stm_km_tot_mld", "stm_prioriteit",
    "stm_aanngeb_ddt", "stm_oh_pg_gst", "stm_geo_gst", "stm_equipm_nr_gst",
    "stm_equipm_soort_gst", "stm_equipm_omschr_gst", "stm_km_van_gst", "stm_km_tot_gst",
    "stm_oorz_groep", "stm_oorz_code", "stm_oorz_tkst", "stm_fh_ddt",
    "stm_fh_status", "stm_sap_storeind_ddt", "stm_tao_indicator", "stm_tao_indicator_vorige",
    "stm_tao_soort_mutatie", "stm_tao_telling_mutatie", "stm_tao_beinvloedbaar_indicator", "stm_sap_melddatum",
    "stm_sap_meldtijd", "stm_contractgeb_mld", "stm_techn_mld", "stm_contractgeb_gst",
    "stm_techn_gst", "stm_aanngeb_dd", "stm_aanngeb_tijd", "stm_aanntpl_dd",
    "stm_aanntpl_tijd", "stm_progfh_in_datum", "stm_progfh_in_tijd", "stm_progfh_in_invoer_dat",
    "stm_progfh_in_invoer_tijd", "stm_progfh_in_duur", "stm_progfh_gw_tijd", "stm_progfh_gw_teller",
    "stm_fh_dd", "stm_fh_tijd", "stm_fh_duur", "stm_sap_storeinddatum",
    "stm_sap_storeindtijd", "stm_oorz_tekst_kort", "stm_pplg_van", "stm_pplg_naar",
    "stm_dstrglp_van", "stm_dstrglp_naar"
]]

#### Opvallend

De data bestaat voornamelijk uit drie soorten gegevens:
- Tekstuele inhoud
- Numerieke klassen
- Datum en tijd

Tijdens het proces wanneer het model wordt gebruikt, zijn niet alle data beschikbaar. Kolommen die eindigen met _gst zijn niet geschikt.

Ook gaan we kijken of er dubbele rijen zijn die er uit gefilterd kunnen worden.

In [43]:
df = df.drop_duplicates()

#### Bruikbare Features

Omdat de dataset enorm en vrij ingewikkeld is. Hebben we samen met de opdrachtgever gezeten om te bepalen welke kolommen relevant zijn, zodat we de rest er uit kunnen filteren.

In [44]:
data = df[[
    'stm_oorz_code', 'stm_sap_melddatum', 'stm_sap_meldtijd', 'stm_geo_mld',
    'stm_aanntpl_tijd', 'stm_fh_tijd', 'stm_techn_mld', 'stm_prioriteit',
    'stm_contractgeb_mld', 'stm_fh_duur', 'stm_progfh_in_duur', 'stm_progfh_in_tijd'
]]

Hier zijn de betekenissen en meetwaardes van de kolommen die we gaan gebruiken:
| Kolom | Betekenis | Meetwaarde |
|-|-|-|
| stm_oorz_code | Oorzaak code | Nominaal |
| stm_sap_melddatum | Datum melding | Ordinaal |
| stm_sap_meldtijd | Tijdstip melding | Ordinaal |
| stm_geo_mld | Geo code melding | Nominaal |
| stm_aanntpl_tijd | Tijdstip aannemer ter plaatse | Ordinaal |
| stm_fh_tijd | Tijdstip functieherstel | Ordinaal |
| stm_techn_mld | Techniekveld melding | Nominaal |
| stm_prioriteit | Prioriteitsindicatie | Ordinaal |
| stm_contractgeb_mld | Contract gebied melding | Nominaal |
| stm_fh_duur | Duur functieherstel | Continue |
| stm_progfh_in_duur | Prognose duur functieherstel | Continue |
| stm_progfh_in_tijd | Prognose tijd functieherstel | Ordinaal |

#### Converteren Tijden

We gaan hier de tijden converteren van HH:MM:SS formaat naar minuten sinds middernacht, zodat we er berekeningen mee kunnen uitvoeren.

In [45]:
# Kolomnamen met tijd in HH:MM:SS formaat
tijd_kolommen = ['stm_sap_meldtijd', 'stm_aanntpl_tijd', 'stm_fh_tijd','stm_progfh_in_tijd']

# Functie om tijdstring om te zetten naar totaal aantal minuten
def convert_time_to_minutes(df, columns):
    for col in columns:
        df.loc[:, col] = pd.to_datetime(df[col], format='%H:%M:%S', errors='coerce')
        df.loc[:, col] = df[col].apply(lambda x: x.hour * 60 + x.minute if pd.notnull(x) else None)
    return df


#### Functie Hersteltijd

We willen de functie hersteltijd gaan voorspellen, dus zullen we moeten kijken hoe deze kolom zich verhoudt to de andere kolommen in de data.

In [46]:
data = convert_time_to_minutes(data, tijd_kolommen)

In [47]:
data[['stm_sap_meldtijd', 'stm_aanntpl_tijd', 'stm_fh_tijd','stm_fh_duur']]

,stm_sap_meldtijd,stm_aanntpl_tijd,stm_fh_tijd,stm_fh_duur
0,NaN,NaN,NaN,NaN
1,540.0,0.0,540.0,0.0
2,755.0,0.0,806.0,51.0
3,1000.0,0.0,1040.0,40.0
4,1350.0,0.0,1356.0,6.0
...,...,...,...,...
908625,486.0,545.0,569.0,83.0
908626,561.0,608.0,644.0,83.0
908627,561.0,608.0,644.0,83.0
908628,855.0,885.0,914.0,19.0


Hieruit blijkt dat functie hersteel duur gelijk aan hersteel tijdstip - meldings tijdstip<br>
ook zien je bij somig storing dat geen infomatie is wanneer aannemer aanweizig was.

In [48]:
len(data[data['stm_fh_duur'] == 0])

160611

In [49]:
len(data.loc[(data["stm_progfh_in_tijd"] == data["stm_fh_tijd"]) & (df["stm_fh_tijd"] != 0)])

291345

In [50]:
len(data[data['stm_fh_duur'] == (data['stm_progfh_in_tijd']-data['stm_aanntpl_tijd'])])

156596

Hieruit kunnen we zien dat 160.000 data geen stroning bij melding.<br>
140.000 data prognose functie herstel tijdstip gelijk zijn aan werkelijk hersteltijd<br>
150.00 data zijn werkelijk FHT gelijk aan tijdstip aannemer te plaats tot prognose functie herstel tijdstip 

in stm_progfh_in_duur zijn str values, we bewaar eerste alle numeric waardes en zet we de type als int

In [51]:
data.loc[:, 'stm_progfh_in_duur'] = pd.to_numeric(data['stm_progfh_in_duur'], errors='coerce')
data.loc[:, 'stm_progfh_in_duur'] = data['stm_progfh_in_duur'].fillna(0).astype(int)

/tmp/ipykernel_2814/3402014665.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.loc[:, 'stm_progfh_in_duur'] = data['stm_progfh_in_duur'].fillna(0).astype(int)


In [52]:
len(data[data['stm_progfh_in_duur'] == (data['stm_progfh_in_tijd']-data['stm_aanntpl_tijd'])])

486618

In [53]:
len(data[data['stm_progfh_in_duur'] == data['stm_fh_duur']])

95215

verder is te zien dat, 120.000 data dat prognose functie herstel tijd gelijk tijdstip aannemer te plaats + prognose functie herstel tijdstip <br>
ook zijn 9.000 data dat prognose duur gelijk aan werkelijk FHT duur

# Target

voor de traget neem we de tijd neem de functie hersteltijd van waarneer de aanmener aanweizig is tot dat weer gereiden mogen worden.<br>
we bewaar eerst alle data dat binnen dag is

In [54]:
data['targetherstel'] = np.where(
    data['stm_fh_tijd'] - data['stm_aanntpl_tijd']>= 0, # als de stm_fh_tijd-stm_aanntpl_tijd positief is
    data['stm_fh_tijd'] - data['stm_aanntpl_tijd'], # dan betekende dat nog op de zefde day is, kun je gelijk van elkaar aftrekken
    data['stm_fh_tijd'] - data['stm_aanntpl_tijd']+1440 # als niet op zelfdde dag is hersteld, dus een neagetief uitkomst moet er +1440 om de juiste tijd te krijgen.
)

/tmp/ipykernel_2814/900286507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['targetherstel'] = np.where(


In [55]:
ranges = {
    "null": (0, 0),
    "tot 15m": (0, 15),
    "15m tot 30m": (15, 30),
    "30m tot 1h": (30, 60),
    "1h tot 2h": (60, 120),
    "2h tot 3h": (120, 180),
    "3h tot 6h": (180, 360),
    "6h tot 8h": (360, 540),
    "8h+": (540, float('inf'))
}

total = len(data)

for label, (low, high) in ranges.items():
    if label == "null":
        count = len(data[data['targetherstel'] == 0])
    else:
        count = len(data[(data['targetherstel'] > low) & (data['targetherstel'] <= high)])
    print(f'score {label}: {count/total:.2%}')


score null: 20.53%
score tot 15m: 21.31%
score 15m tot 30m: 12.51%
score 30m tot 1h: 15.28%
score 1h tot 2h: 12.22%
score 2h tot 3h: 4.81%
score 3h tot 6h: 5.22%
score 6h tot 8h: 1.74%
score 8h+: 6.38%


we zien dat dat 20% van de meldingen geen probleem zijn dat de spoor op stil staat
21% dat onder de 15 miuten is en 13% tussen 15-30 min
voor meldingen dat de spoor boven de acht uurr stil staan zij er maar 6%

In [56]:
len(data[data['stm_progfh_in_duur'] == data['targetherstel']])

196142

# Filter

filter data's dat stm_fh_duur onder de 15 min en data's boven de 8 uur,het is niet belangrjik in deze project.<br>
we nemen data's van 15min tot 8uur, zo data de model niet telaag gaat onderschaten.

In [57]:
data = data.loc[ (data['targetherstel']>15) & (data['targetherstel']<=540)]

Filter van grognose duur, er mag geen rare waardes als 0 of 99999999 in staan

In [58]:
data = data.loc[(data['stm_progfh_in_duur']>0) & (data['stm_progfh_in_duur']<=1440)]

Filter voor menselijk fouten waar de waarders achteraf is door de aannemer ingevoerd zijn.

In [59]:
data = data.loc[(data["stm_progfh_in_tijd"] != data["stm_fh_tijd"])]

In [60]:
data = data[data['stm_progfh_in_duur'] != data['stm_fh_duur']]

In [61]:
data = data[data['stm_progfh_in_duur'] != data['targetherstel']]

# datum reparatie

In [62]:
data = data.copy()
#Zet de kolom om naar datetime,vervang de datum door de dag van het jaar
data.loc[:, 'stm_sap_melddatum'] = pd.to_datetime(data['stm_sap_melddatum'], format='%d/%m/%Y').dt.dayofyear

# prioriteit

In [63]:
data=data.loc[(data['stm_prioriteit']!=8)&(data['stm_prioriteit']!=9)]

In [64]:
data

,stm_oorz_code,stm_sap_melddatum,stm_sap_meldtijd,stm_geo_mld,stm_aanntpl_tijd,stm_fh_tijd,stm_techn_mld,stm_prioriteit,stm_contractgeb_mld,stm_fh_duur,stm_progfh_in_duur,stm_progfh_in_tijd,targetherstel
98626,215.0,201,545.0,119.0,30.0,240.0,NaN,2.0,NaN,1012.0,270,300.0,210.0
98627,215.0,201,545.0,119.0,30.0,240.0,NaN,2.0,NaN,1012.0,270,300.0,210.0
144452,215.0,279,749.0,63.0,788.0,810.0,S,4.0,26.0,53.0,31,819.0,22.0
144453,215.0,279,749.0,63.0,788.0,810.0,S,4.0,26.0,53.0,31,819.0,22.0
144457,215.0,279,1012.0,62.0,583.0,632.0,S,4.0,25.0,1051.0,41,624.0,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
908625,215.0,131,486.0,560.0,545.0,569.0,NaN,5.0,NaN,83.0,30,575.0,24.0
908626,218.0,131,561.0,468.0,608.0,644.0,S,2.0,4.0,83.0,52,660.0,36.0
908627,218.0,131,561.0,468.0,608.0,644.0,S,2.0,4.0,83.0,52,660.0,36.0
908628,135.0,233,855.0,102.0,885.0,914.0,NaN,2.0,NaN,19.0,75,960.0,29.0


## Modellen

### Baseline

We neem RMSE van prognose tijd duur met de traget als de baseline voor de Linearrregressie model.

In [65]:
np.sqrt(mean_squared_error(data['stm_progfh_in_duur'], data['targetherstel']))

np.float64(198.37210509252483)

Naast de RMSE waarde, wanneer kan we zeggen of een verspelling ook een goede verspelling is?<br> Als de verspelling target herstel tijd kan bedekken met een maximale verschil van 15 minuten.

In [66]:
tolerance = 15

correct = (data['stm_progfh_in_duur'] > data['targetherstel']) & \
          ((data['stm_progfh_in_duur'] - data['targetherstel']) <= tolerance)

accuracy = np.mean(correct)

print(f"Accuracy (overschatting binnen {tolerance} minuten): {accuracy:.2%}")

Accuracy (overschatting binnen 15 minuten): 35.24%


In [67]:
np.abs(data['stm_progfh_in_duur'] - data['targetherstel']) 

98626     60.0
98627     60.0
144452     9.0
144453     9.0
144457     8.0
          ... 
908625     6.0
908626    16.0
908627    16.0
908628    46.0
908629    46.0
Length: 242010, dtype: object

In [68]:
len(data[data['stm_progfh_in_duur'] > data['targetherstel']])/len(data)

0.8232882938721541

Uit data blijk 35.24% van data van de prognose zal goede verspelling zijn, waarbij 82% van alle data altijd de targetherstel tijd kunt bedekken is.<br>
Het is belangrijk dat de voorspelling beter overschat dan onderschat in de project.

Dus wanner is het model beter dan nu?<br>
1.Als RMSE kleiner wordt<br>
2.Als verspelling vanm  met marge van 15 minuten het Target kunt bedekken<br>
3.Als de totale verspelling een betere dekkingsgraad heeft.

### Linear Regression

In [69]:
# zet alle Nan naar 0

In [70]:
data = data.fillna(0)

/tmp/ipykernel_2814/711981726.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.fillna(0)


In [71]:
def Train_per_categorical(data):
    data = data.copy()  # Voorkom SettingWithCopyWarning

    # Categorische kolommen die één voor één toegevoegd worden
    categorical_cols = ['stm_oorz_code', 'stm_geo_mld', 'stm_contractgeb_mld', 'stm_techn_mld', 'stm_prioriteit']

    # Numerieke kolommen die altijd in het model zitten
    fixed_numeric_cols = ['stm_sap_meldtijd','stm_sap_melddatum','stm_aanntpl_tijd', 'stm_progfh_in_duur']

    # Eerst de baseline (alleen vaste numerieke features)
    print("Baseline model: alleen numerieke features")
    X_base = data[fixed_numeric_cols]
    y = data['targetherstel']

    X_train, X_test, y_train, y_test = train_test_split(X_base, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f"[Baseline] R² = {r2:.3f}, RMSE = {rmse:.2f}")

    # Accuracy en coverage
    tolerance = 15
    correct = (data['stm_progfh_in_duur'] > data['targetherstel']) & \
              ((data['stm_progfh_in_duur'] - data['targetherstel']) <= tolerance)
    accuracy = np.mean(correct)
    coverage = np.mean(data['stm_progfh_in_duur'] > data['targetherstel'])
    print(f"[Baseline] Accuracy (±{tolerance} min overschatting): {accuracy:.2%}")
    print(f"[Baseline] Dekkingsgraad: {coverage:.2%}")

    # Vervolgens per categorische kolom apart toevoegen
    for col in categorical_cols:
        print(f"\nModel trainen met categorische kolom: '{col}' + vaste numerieke features")

        data[col] = data[col].astype(str)
        X_cat = pd.get_dummies(data[[col]], drop_first=True)

        X = pd.concat([data[fixed_numeric_cols], X_cat], axis=1)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))

        print(f"R² = {r2:.3f}, RMSE = {rmse:.2f}")

        # Accuracy & coverage opnieuw berekenen (zelfde formule)
        correct = (data['stm_progfh_in_duur'] > data['targetherstel']) & \
                  ((data['stm_progfh_in_duur'] - data['targetherstel']) <= tolerance)
        accuracy = np.mean(correct)
        coverage = np.mean(data['stm_progfh_in_duur'] > data['targetherstel'])

        print(f"Accuracy (±{tolerance} min overschatting): {accuracy:.2%}")
        print(f"Dekkingsgraad: {coverage:.2%}")



In [72]:
Train_per_categorical(data)

Baseline model: alleen numerieke features
[Baseline] R² = 0.127, RMSE = 76.48
[Baseline] Accuracy (±15 min overschatting): 35.24%
[Baseline] Dekkingsgraad: 82.33%

Model trainen met categorische kolom: 'stm_oorz_code' + vaste numerieke features
R² = 0.169, RMSE = 74.60
Accuracy (±15 min overschatting): 35.24%
Dekkingsgraad: 82.33%

Model trainen met categorische kolom: 'stm_geo_mld' + vaste numerieke features
R² = 0.134, RMSE = 76.15
Accuracy (±15 min overschatting): 35.24%
Dekkingsgraad: 82.33%

Model trainen met categorische kolom: 'stm_contractgeb_mld' + vaste numerieke features
R² = 0.133, RMSE = 76.19
Accuracy (±15 min overschatting): 35.24%
Dekkingsgraad: 82.33%

Model trainen met categorische kolom: 'stm_techn_mld' + vaste numerieke features
R² = 0.135, RMSE = 76.12
Accuracy (±15 min overschatting): 35.24%
Dekkingsgraad: 82.33%

Model trainen met categorische kolom: 'stm_prioriteit' + vaste numerieke features
R² = 0.130, RMSE = 76.34
Accuracy (±15 min overschatting): 35.24%
Dekk

## Decision Tree

In [73]:
def train_decision_tree(data):
    """Train verbeterd Decision Tree model met hyperparameter tuning"""
    data = data.copy()

    # Alle feature categorieën
    numeric_cols = ['stm_fh_duur', 'stm_progfh_in_duur', 'stm_sap_melddatum', 'stm_sap_meldtijd',
                   'stm_aanntpl_tijd', 'stm_fh_tijd', 'stm_progfh_in_tijd', 'stm_prioriteit']
    categorical_cols = ['stm_oorz_code', 'stm_geo_mld', 'stm_techn_mld', 'stm_contractgeb_mld']

    # Feature engineering: voeg afgeleide features toe
    data['tijd_verschil_meld_aanntpl'] = data['stm_aanntpl_tijd'] - data['stm_sap_meldtijd']
    data['tijd_verschil_aanntpl_fh'] = data['stm_fh_tijd'] - data['stm_aanntpl_tijd']
    data['progfh_vs_werkelijk'] = data['stm_progfh_in_duur'] / (data['stm_fh_duur'] + 1)
    data['prognose_afwijking'] = np.abs(data['stm_progfh_in_duur'] - data['stm_fh_duur'])

    # Voeg tijdsgerelateerde features toe
    data['is_werkdag'] = (data['stm_sap_melddatum'] % 7 < 5).astype(int)
    data['uur_van_dag'] = (data['stm_sap_meldtijd'] // 60).astype(int)
    data['is_spitsuur'] = ((data['uur_van_dag'] >= 7) & (data['uur_van_dag'] <= 9) |
                          (data['uur_van_dag'] >= 17) & (data['uur_van_dag'] <= 19)).astype(int)

    # Update numeric columns met nieuwe features
    numeric_cols.extend(['tijd_verschil_meld_aanntpl', 'tijd_verschil_aanntpl_fh',
                        'progfh_vs_werkelijk', 'prognose_afwijking', 'is_werkdag',
                        'uur_van_dag', 'is_spitsuur'])

    # Prepare features
    X_numeric = data[numeric_cols]

    # One-hot encode categorische features met frequentie filtering
    X_categorical = pd.DataFrame()
    for col in categorical_cols:
        # Filter lage frequentie categorieën
        value_counts = data[col].value_counts()
        frequent_values = value_counts[value_counts >= 50].index
        data[col] = data[col].where(data[col].isin(frequent_values), 'Other')

        cat_encoded = pd.get_dummies(data[col].astype(str), prefix=col, drop_first=True)
        X_categorical = pd.concat([X_categorical, cat_encoded], axis=1)

    # Combineer alle features
    X = pd.concat([X_numeric, X_categorical], axis=1)
    y = data['targetherstel']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Verbeterd Decision Tree model met betere hyperparameters
    model = DecisionTreeRegressor(
        random_state=42,
        max_depth=20,           # Dieper voor meer complexiteit
        min_samples_split=20,   # Hoger voor regularisatie
        min_samples_leaf=10,    # Hoger voor regularisatie
        max_features=0.7,       # Meer features beschikbaar
        min_impurity_decrease=0.001,  # Verminder overfitting
        ccp_alpha=0.01          # Cost complexity pruning
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluatie
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("Verbeterd Decision Tree model:")
    print(f"R² = {r2:.3f}, RMSE = {rmse:.2f}")
    print(f"Aantal features: {X.shape[1]}")

    # Test verschillende toleranties
    for tolerance in [10, 15, 20, 30]:
        correct = (y_pred >= y_test) & ((y_pred - y_test) <= tolerance)
        accuracy = np.mean(correct)
        coverage = np.mean(y_pred >= y_test)
        print(f"Tolerance ±{tolerance} min - Accuracy: {accuracy:.2%}, Coverage: {coverage:.2%}")

    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 10 belangrijkste features:")
    print(feature_importance.head(10))

    return model, X.columns.tolist(), y_test, y_pred


In [74]:
model, feature_names, test, predict = train_decision_tree(data)

Verbeterd Decision Tree model:
R² = 0.999, RMSE = 1.97
Aantal features: 495
Tolerance ±10 min - Accuracy: 50.58%, Coverage: 50.71%
Tolerance ±15 min - Accuracy: 50.67%, Coverage: 50.71%
Tolerance ±20 min - Accuracy: 50.69%, Coverage: 50.71%
Tolerance ±30 min - Accuracy: 50.70%, Coverage: 50.71%

Top 10 belangrijkste features:
                       feature  importance
9     tijd_verschil_aanntpl_fh    0.984686
1           stm_progfh_in_duur    0.005804
0                  stm_fh_duur    0.004634
4             stm_aanntpl_tijd    0.002707
22         stm_oorz_code_143.0    0.001300
5                  stm_fh_tijd    0.000424
8   tijd_verschil_meld_aanntpl    0.000184
10         progfh_vs_werkelijk    0.000179
14                 is_spitsuur    0.000041
6           stm_progfh_in_tijd    0.000029


### Random Forest

In [75]:
def train_random_forest(data):
    """Train verbeterd Random Forest model met hyperparameter tuning"""
    data = data.copy()

    # Alle feature categorieën
    numeric_cols = ['stm_fh_duur', 'stm_progfh_in_duur', 'stm_sap_melddatum', 'stm_sap_meldtijd',
                   'stm_aanntpl_tijd', 'stm_fh_tijd', 'stm_progfh_in_tijd', 'stm_prioriteit']
    categorical_cols = ['stm_oorz_code', 'stm_geo_mld', 'stm_techn_mld', 'stm_contractgeb_mld']

    # Feature engineering: voeg afgeleide features toe
    data['tijd_verschil_meld_aanntpl'] = data['stm_aanntpl_tijd'] - data['stm_sap_meldtijd']
    data['tijd_verschil_aanntpl_fh'] = data['stm_fh_tijd'] - data['stm_aanntpl_tijd']
    data['progfh_vs_werkelijk'] = data['stm_progfh_in_duur'] / (data['stm_fh_duur'] + 1)
    data['prognose_afwijking'] = np.abs(data['stm_progfh_in_duur'] - data['stm_fh_duur'])

    # Voeg tijdsgerelateerde features toe
    data['is_werkdag'] = (data['stm_sap_melddatum'] % 7 < 5).astype(int)
    data['uur_van_dag'] = (data['stm_sap_meldtijd'] // 60).astype(int)
    data['is_spitsuur'] = ((data['uur_van_dag'] >= 7) & (data['uur_van_dag'] <= 9) |
                          (data['uur_van_dag'] >= 17) & (data['uur_van_dag'] <= 19)).astype(int)

    # Interactie features
    data['prioriteit_x_progduur'] = data['stm_prioriteit'] * data['stm_progfh_in_duur']
    data['weekdag_x_uur'] = data['is_werkdag'] * data['uur_van_dag']

    # Update numeric columns met nieuwe features
    numeric_cols.extend(['tijd_verschil_meld_aanntpl', 'tijd_verschil_aanntpl_fh',
                        'progfh_vs_werkelijk', 'prognose_afwijking', 'is_werkdag',
                        'uur_van_dag', 'is_spitsuur', 'prioriteit_x_progduur', 'weekdag_x_uur'])

    # Prepare features
    X_numeric = data[numeric_cols]

    # One-hot encode categorische features met frequentie filtering
    X_categorical = pd.DataFrame()
    for col in categorical_cols:
        # Filter lage frequentie categorieën
        value_counts = data[col].value_counts()
        frequent_values = value_counts[value_counts >= 100].index  # Hogere drempel voor RF
        data[col] = data[col].where(data[col].isin(frequent_values), 'Other')

        cat_encoded = pd.get_dummies(data[col].astype(str), prefix=col, drop_first=True)
        X_categorical = pd.concat([X_categorical, cat_encoded], axis=1)

    # Combineer alle features
    X = pd.concat([X_numeric, X_categorical], axis=1)
    y = data['targetherstel']

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Verbeterd Random Forest model
    model = RandomForestRegressor(
        n_estimators=500,       # Meer bomen voor betere prestaties
        random_state=42,
        max_depth=25,           # Dieper voor meer complexiteit
        min_samples_split=10,   # Lager voor meer flexibiliteit
        min_samples_leaf=5,     # Lager voor meer flexibiliteit
        max_features=0.6,       # Optimale feature subset
        bootstrap=True,
        n_jobs=-1,
        oob_score=True,         # Out-of-bag score voor extra evaluatie
        max_samples=0.8,        # Sample subsets voor diversiteit
        min_impurity_decrease=0.0005  # Lichte regularisatie
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # Evaluatie
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print("Verbeterd Random Forest model:")
    print(f"R² = {r2:.3f}, RMSE = {rmse:.2f}")
    print(f"OOB Score: {model.oob_score_:.3f}")
    print(f"Aantal features: {X.shape[1]}")

    # Test verschillende toleranties
    for tolerance in [10, 15, 20, 30]:
        correct = (y_pred >= y_test) & ((y_pred - y_test) <= tolerance)
        accuracy = np.mean(correct)
        coverage = np.mean(y_pred >= y_test)
        print(f"Tolerance ±{tolerance} min - Accuracy: {accuracy:.2%}, Coverage: {coverage:.2%}")

    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)

    print("\nTop 10 belangrijkste features:")
    print(feature_importance.head(10))

    return model, X.columns.tolist(), y_test, y_pred


In [ ]:
rf_model, feature_names, test, predict = train_random_forest(data)